- Dict with top models for each horizon
- Trainflow where number of runs equals horizon
- Append each training day to master list for performance tracking?
- Pull in performance over last n_days (120 min?, 30 and 10?)
    - Focused on precision
- Also ensemble scoring

In [91]:
import performance_flow
import importlib
importlib.reload(performance_flow)
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
#df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
#past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
#past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
#past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
#cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
#df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Parse Top Models per Horizon

In [87]:
import importlib
importlib.reload(performance_flow)
# ---------- #
# UPDATE ME  #
# ---------- #
min_th = 0.55
cov_th = 0.75
horizons = [2]#[2, 5, 10, 20, 30]

for r in horizons:
    
    results_file_name = f"h{r}_performance_all.csv" # Match prior cell saved as file name horizon_2_baseline_new
    return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
    perf_df = perf_df.rename(columns={"feature_set": "features"})
    composite_score = performance_flow.run_performance(perf_df[perf_df['horizon'] == r], min_th, cov_th)
    bucket_df = performance_flow.bucket_scores(df_daily, perf_df[perf_df['horizon'] == r], returns, min_th, keys)

    top_10_r = (
    composite_score.sort_values("composite", ascending=False)
    .drop_duplicates(subset=["features"], keep="first").head(10).copy())

    keys = ["horizon", "features", "train_years", "min_feats", "pi_size", "pi_handling", "model"]

    # Ensure dtypes match so the join actually hits
    for df in (top_10_r, perf_df):

        df["horizon"] = r
        df["features"] = df["features"].astype(str)
        df["model"]       = df["model"].astype(str)
        df["pi_size"]     = df["pi_size"]
        df["pi_handling"]     = df["pi_handling"].astype(str)
        df["train_years"] = df["train_years"].astype(int)
        df["min_feats"]   = df["min_feats"].astype(int)

    # Filter master predictions to only rows matching one of the 10 configs
    pred_filtered = perf_df.merge(top_10_r[keys].drop_duplicates(), on=keys, how="inner")
    #print(len(pred_filtered))
    pred_filtered.to_csv(f"h{r}_top10_raw.csv", index=False)

In [95]:
from itertools import chain
import deployment_flow
import importlib
importlib.reload(deployment_flow)

def resolve_feature_cols(feature_set_name: str, features_dict: dict, sep: str = "-") -> list[str]:

    # --- Kitchen sink case ---
    if feature_set_name == "kitch_sink":
        all_cols = chain.from_iterable(features_dict.values())
        # dedupe preserve order
        seen = set()
        out = []
        for c in all_cols:
            if c not in seen:
                seen.add(c)
                out.append(c)
        return out

    # --- Normal composite case ---
    parts = feature_set_name.split(sep)

    cols = []
    for p in parts:
        if p not in features_dict:
            raise KeyError(f"{p} not in features_dict")
        cols.append(features_dict[p])

    # flatten + dedupe
    seen = set()
    out = []
    for c in chain.from_iterable(cols):
        if c not in seen:
            seen.add(c)
            out.append(c)

    return out

h=[2]

for r in h:
    
    df = pd.read_csv(f"h{r}_top10_raw.csv")

    df["feature_cols"] = df["features"].apply(lambda x: resolve_feature_cols(x, feature_dict))

    grain_cols = ["horizon","features","train_years","min_feats","pi_size","model","pi_handling"]

    max_train = (
        df.groupby(grain_cols, as_index=False)["test_start"]
        .max()
        .rename(columns={"test_start": "max_test_start"})
    )

    top_10_unique = (
        df[grain_cols].drop_duplicates(subset=grain_cols, keep="first")
        .merge(df[grain_cols + ["feature_cols"]].drop_duplicates(subset=grain_cols), on=grain_cols, how="left")
        .merge(max_train, on=grain_cols, how="left")   # <-- this is the missing piece
    )
    
    for row in top_10_unique.itertuples(index=False):

        target_horizon = row.horizon
        pi_handling    = 'run_separately' #row.pi_handling
        feature_cols   = row.feature_cols   # list-of-cols wrapped in a list
        list_name      = row.features
        train_year     = row.train_years
        pi_year        = row.pi_size
        min_feat       = row.min_feats
        max_test_start = row.max_test_start
        days_assessed  = len(df_daily.iloc[r:][df_daily['Date'] > max_test_start].copy())
        groups = list_name.split("-")

        model = XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)
        model_name = "xgboost-3"

        print(f"{target_horizon} | {pi_handling} | {list_name} | {train_year} | {pi_year} | {min_feat} | {days_assessed}")
        results_df = deployment_flow.run_deploy_flow(days_assessed, r, pi_handling, feature_cols, df_daily, model_name, model,
                        train_year, pi_year, min_feat, list_name, feature_dict, groups)

        #performance_df = pd.read_csv(f"h{r}_baseline.csv")
        #df_concat = pd.concat([performance_df, results_df], ignore_index=True)    
        #df_concat.to_csv(f"h{r}_baseline_v2.csv", index=False)

2 | run_separately | ma_lag-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | ma_rel-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | ma_sma-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | ma_num-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | rsi_macd-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | volu-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | atr_adxvola-vix_skew | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | atr_adxvola-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | vix_skew-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately
2 | run_separately | experimental_slope-past_return | 5 | 1.5 | 4 | 0
Running for horizon 2 | run_separately


- Train through most recent actualized data (Done)
- Append all actuals to raw files
- Recalc performance 
- Predict on new horizon (Done)
- Show prediction (Done)
- Bring in performance
- Ensemble prediction

In [80]:
import importlib
importlib.reload(performance_flow)
# ---------- #
# UPDATE ME  #
# ---------- #
min_th = 0.55
cov_th = 0.75
horizons = [2]#[2, 5, 10, 20, 30]

for r in horizons:
    
    keys = ["horizon", "features", "train_years", "min_feats", "pi_size", "pi_handling", "model"]

    results_file_name = f"h{r}_performance_all.csv" # Match prior cell saved as file name horizon_2_baseline_new
    return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
    perf_df = perf_df.rename(columns={"feature_set": "features"})
    composite_score = performance_flow.run_performance(perf_df[perf_df['horizon'] == r], min_th, cov_th)
    bucket_df = performance_flow.bucket_scores(df_daily, perf_df[perf_df['horizon'] == r], returns, min_th, keys)

    top_10_r = (
    composite_score.sort_values("composite", ascending=False)
    .drop_duplicates(subset=["features"], keep="first").head(10).copy())

    # Ensure dtypes match so the join actually hits
    for df in (top_10_r, perf_df):

        df["horizon"] = r
        df["features"] = df["features"].astype(str)
        df["model"]       = df["model"].astype(str)
        df["pi_size"]     = df["pi_size"]
        df["pi_handling"]     = df["pi_handling"].astype(str)
        df["train_years"] = df["train_years"].astype(int)
        df["min_feats"]   = df["min_feats"].astype(int)

    # Filter master predictions to only rows matching one of the 10 configs
    #pred_filtered = perf_df.merge(top_10_r[keys].drop_duplicates(), on=keys, how="inner")
    #print(len(pred_filtered))
    #pred_filtered.to_csv(f"h{r}_top10_raw.csv", index=False)
        
bucket_df

,horizon,model,train_years,features,pi_size,min_feats,n_1_c,n_-1_c,acc_1_c,acc_-1_c,bal_acc_1_o,bal_acc_2_o,bal_acc_3_o,bal_acc_3p_o,wba_open,wba_close
23,2,xgboost-3,5,ma_rel-past_return,1.5,4,20.0,20.0,0.80,0.70,0.76,0.76,0.72,0.82,0.76,0.75
3,2,xgboost-3,5,experimental_slope-past_return,1.5,4,20.0,20.0,0.65,0.60,0.77,0.82,0.73,0.75,0.77,0.74
40,2,xgboost-3,5,vix_skew-past_return,1.5,4,20.0,19.0,0.70,0.63,0.72,0.87,0.73,0.72,0.76,0.74
30,2,xgboost-3,5,ma_sma-past_return,1.5,4,20.0,19.0,0.75,0.68,0.71,0.79,0.83,0.73,0.76,0.73
15,2,xgboost-3,5,ma_num-past_return,1.5,4,20.0,20.0,0.70,0.55,0.77,0.76,0.77,0.69,0.76,0.72
9,2,xgboost-3,5,ma_lag-past_return,1.5,4,20.0,19.0,0.75,0.58,0.77,0.73,0.79,0.73,0.75,0.72
1,2,xgboost-3,5,atr_adxvola-past_return,1.5,4,19.0,18.0,0.68,0.61,0.78,0.66,0.80,0.78,0.75,0.71
36,2,xgboost-3,5,rsi_macd-past_return,1.5,4,20.0,19.0,0.65,0.58,0.72,0.76,0.83,0.70,0.75,0.71
43,2,xgboost-3,5,volu-past_return,1.5,4,19.0,19.0,0.79,0.53,0.76,0.71,0.82,0.73,0.75,0.71
13,2,xgboost-3,5,ma_num-atr_adxvola,1.5,4,20.0,20.0,0.75,0.35,0.60,0.56,0.48,0.50,0.55,0.53


In [118]:
import deployment_flow
import importlib
importlib.reload(deployment_flow)

h = [2]
master_results = []

for r in h:
    
    days_assessed = len(df_daily[df_daily[f"Return_{r}"].isna()])

    df = pd.read_csv(f"h{r}_top10_raw.csv")

    df["feature_cols"] = df["features"].apply(lambda x: resolve_feature_cols(x, feature_dict))

    grain_cols = ["horizon","features","train_years","min_feats","pi_size","model","pi_handling"]

    top_10_unique = (
        df[grain_cols].drop_duplicates(subset=grain_cols, keep="first")
        .merge(df[grain_cols + ["feature_cols"]].drop_duplicates(subset=grain_cols), on=grain_cols, how="left"))
    
    for row in top_10_unique.itertuples(index=False):

        target_horizon = row.horizon
        pi_handling    = 'run_separately' #row.pi_handling
        type           = 'New_Predict'
        feature_cols   = row.feature_cols   # list-of-cols wrapped in a list
        list_name      = row.features
        train_year     = row.train_years
        pi_year        = row.pi_size
        min_feat       = row.min_feats
        groups = list_name.split("-")

        model = XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)
        model_name = "xgboost-3"

        print(f"{target_horizon} | {pi_handling} | {list_name} | {train_year} | {pi_year} | {min_feat} | {days_assessed}")
        results_df = deployment_flow.run_deploy_flow(days_assessed, r, pi_handling, feature_cols, df_daily, model_name, model,
                        train_year, pi_year, min_feat, list_name, feature_dict, groups, type)
        
        master_results.append(results_df)

master_results_df = pd.concat(master_results, ignore_index=True)

2 | run_separately | ma_lag-past_return | 5 | 1.5 | 4 | 2
Running new predictions for horizon 2 | run_separately
ma_lag: 60 | 4 | ['SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'SMA_10_Lag50_min', 'SMA_25_Lag200_min']
past_return: 14 | 6 | ['Past_Return%_10', 'Past_Return%_2', 'Past_Return%_20', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5']
Run 1/2 | Train: 2021-03-26 → 2026-02-10 | Test: 2026-02-13 → 2026-02-13 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 4)
ma_lag: 60 | 6 | ['SMA_10_Lag100_min', 'SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'SMA_10_Lag50_min', 'SMA_25_Lag200_min', 'SMA_25_Lag25_min']
past_return: 14 | 5 | ['Past_Return%_1', 'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5']
Run 2/2 | Train: 2021-03-25 → 2026-02-09 | Test: 2026-02-12 → 2026-02-12 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 4)
2 | run_separately | ma_rel-past_return | 5 | 1.5 | 4 | 2
Running new predictions for horizon 2 | run_separately
ma_rel: 20 | 9 | ['Close_Rel_

# Archived

In [158]:
def normalize_csvs():
    
    for r in returns:
        
        past_performance = pd.read_csv(f"h{r}_baseline.csv") # Match prior cell saved as file name horizon_2_baseline_new
        """
        past_performance["features"] = (
        past_performance["feature_set"]
            .str.replace("_ba$", "", regex=True)
            .str.replace("-baseline$", "", regex=True)
            .str.replace("-pr$", "-past_return", regex=True)
            .str.replace("past_ret_cols", "past_return", regex=False)
        )
        """

        past_performance["pi_handling"] = (
        past_performance["pi_size"]
        .str.replace("1.5-r", "run_separately", regex=False)
        .str.replace("1.5", "include_new", regex=False)
        )

        past_performance["pi_size"] = 1.5
        past_performance.to_csv(f"h{r}_baseline.csv", index=False)